In [1]:
import psycopg2
from datetime import datetime

# PostgreSQL database credentials
DB_CONFIG = {
    "dbname": "rag_db",
    "user": "user",
    "password": "password",
    "host": "192.168.0.34",
    "port": 5432
}

def fetch_high_similarity_tickets():
    """
    Fetch high-similarity tickets and associated quantum numbers.
    """
    query = """
    WITH high_similarity_tickets AS (
        SELECT 
            wn.id AS winning_id,
            pts.id AS ticket_id,
            wn.draw_date,
            pts.generated_at AS ticket_generated_at,
            wn.source AS winning_source,
            pts.source AS ticket_source,
            pts.embedding <-> ARRAY[
                wn.white_ball_1 / 69.0,
                wn.white_ball_2 / 69.0,
                wn.white_ball_3 / 69.0,
                wn.white_ball_4 / 69.0,
                wn.white_ball_5 / 69.0,
                wn.powerball / 26.0
            ]::vector AS similarity,
            pts.process_id
        FROM 
            winning_numbers wn
        JOIN 
            powerball_ticket_sessions pts
        ON 
            TRUE
        ORDER BY 
            similarity ASC
        LIMIT 6
    ),
    last_quantum_numbers AS (
        SELECT 
            id AS quantum_id,
            generated_at AS quantum_generated_at,
            process_id
        FROM 
            powerball_ticket_sessions
        WHERE 
            source = 'quantum'
    )
    SELECT 
        hst.winning_id,
        hst.ticket_id,
        hst.draw_date,
        hst.ticket_generated_at,
        lqn.quantum_generated_at,
        hst.similarity,
        hst.process_id
    FROM 
        high_similarity_tickets hst
    CROSS JOIN LATERAL (
        SELECT 
            * 
        FROM 
            last_quantum_numbers lqn
        WHERE 
            lqn.process_id = hst.process_id
            AND lqn.quantum_generated_at <= hst.ticket_generated_at
        ORDER BY 
            lqn.quantum_generated_at DESC
        LIMIT 1
    ) lqn;
    """
    try:
        # Connect to the database
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()

        # Execute the query
        cursor.execute(query)
        results = cursor.fetchall()

        # Close the connection
        cursor.close()
        conn.close()

        return results
    except Exception as e:
        print(f"Database error: {e}")
        return []

def calculate_proximity(results):
    """
    Calculate the proximity in minutes for high-similarity tickets.
    """
    proximity_data = []
    for row in results:
        winning_id, ticket_id, draw_date, ticket_generated_at, quantum_generated_at, similarity, process_id = row

        # Calculate proximity in minutes
        proximity_minutes = (ticket_generated_at - quantum_generated_at).total_seconds() / 60

        # Store the result
        proximity_data.append({
            "winning_id": winning_id,
            "ticket_id": ticket_id,
            "draw_date": draw_date,
            "ticket_generated_at": ticket_generated_at,
            "quantum_generated_at": quantum_generated_at,
            "proximity_minutes": proximity_minutes,
            "similarity": similarity,
            "process_id": process_id
        })

    return proximity_data

def main():
    # Fetch high-similarity tickets
    results = fetch_high_similarity_tickets()
    if not results:
        print("No results found.")
        return

    # Calculate proximity
    proximity_data = calculate_proximity(results)

    # Print the results
    print("Proximity Results:")
    for data in proximity_data:
        print(f"Winning ID: {data['winning_id']}, Ticket ID: {data['ticket_id']}, "
              f"Proximity Minutes: {data['proximity_minutes']:.2f}, Similarity: {data['similarity']:.5f}, "
              f"Process ID: {data['process_id']}")

if __name__ == "__main__":
    main()


Proximity Results:
Winning ID: 9, Ticket ID: 358, Proximity Minutes: 15.59, Similarity: 0.03834, Process ID: 337816
Winning ID: 9, Ticket ID: 911, Proximity Minutes: 19.61, Similarity: 0.05431, Process ID: 337816
Winning ID: 10, Ticket ID: 601, Proximity Minutes: 50.22, Similarity: 0.06149, Process ID: 337816
Winning ID: 1, Ticket ID: 2173, Proximity Minutes: 3.75, Similarity: 0.07100, Process ID: 460803
Winning ID: 7, Ticket ID: 1085, Proximity Minutes: 0.72, Similarity: 0.07253, Process ID: 337816
